In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import StructuredOutputParser
from langchain.output_parsers.json import parse_json_markdown
from langchain.chains import LLMChain
from aux import PromptUtility, DBUtility

with open('openai_key.txt', encoding='utf-8') as f:
    openai_key = f.readlines()[0]

db = DBUtility(database_path='/Users/francisco/Library/CloudStorage/OneDrive-Personal/Francisco/GitHub/GenerativeDebunking/database/CARDS.db')
chat = ChatOpenAI(temperature=0.0, openai_api_key=openai_key)

In [2]:
# Chains

def first_chain(argument):
    # prompt
    first_prompt = ChatPromptTemplate.from_template(PromptUtility.get_first_template())
    # chain
    fc = LLMChain(llm=chat, prompt=first_prompt)
    # parser
    first_response_schemas = [
    ResponseSchema(name="P#", description="premise of the argument"),
    ResponseSchema(name="C", description="conclusion of the argument")]
    first_output_parser = StructuredOutputParser.from_response_schemas(first_response_schemas)
    # run
    output_content = fc.run({'argument':argument, 
                            'delimiter':PromptUtility.get_delimiter, 
                            'example':PromptUtility.get_first_example, 
                            'format_instructions':first_output_parser.get_format_instructions()})
    # parse output
    output = parse_json_markdown(output_content)
    return output

def second_chain(first_chain_output, argument):
    # prompt
    second_prompt = ChatPromptTemplate.from_template(PromptUtility.get_second_template())
    # chain
    sc = LLMChain(llm=chat, prompt=second_prompt)
    # parser
    second_response_schemas = [
    ResponseSchema(name="valid", description="TRUE or FALSE"),
    ResponseSchema(name="reason", description="Explain your reasoning")]
    second_output_parser = StructuredOutputParser.from_response_schemas(second_response_schemas)
    # run
    output_content = sc.run({'argument':PromptUtility.print_dict(first_chain_output, argument), 
                            'delimiter':PromptUtility.get_delimiter(), 
                            'example':PromptUtility.get_second_example(), 
                            'format_instructions':second_output_parser.get_format_instructions()})
    # parse output
    output = parse_json_markdown(output_content)
    return output

def third_chain(first_chain_output, argument):
    # prompt
    third_prompt = ChatPromptTemplate.from_template(PromptUtility.get_third_template())
    # chain
    tc = LLMChain(llm=chat, prompt=third_prompt)
    # parser
    third_response_schemas = [
    ResponseSchema(name="HP", description="Hidden premise"),]
    third_output_parser = StructuredOutputParser.from_response_schemas(third_response_schemas)
    # run
    output_content = tc.run({'argument':PromptUtility.print_dict(first_chain_output, argument), 
                                'delimiter':PromptUtility.get_delimiter(), 
                                'example':PromptUtility.get_third_example(), 
                                'format_instructions':third_output_parser.get_format_instructions()})
    # parse output
    output = parse_json_markdown(output_content)
    return output

def fourth_chain(premise):
    # prompt
    fourth_prompt = ChatPromptTemplate.from_template(PromptUtility.get_fourth_template())
    # chain
    fc = LLMChain(llm=chat, prompt=fourth_prompt)
    # parser
    fourth_output_parser = CommaSeparatedListOutputParser()
    # run
    output_content = fc.run({'argument':premise,
                             'delimiter':PromptUtility.get_delimiter(),
                             'table':PromptUtility.get_table()})
    # parse output
    output = fourth_output_parser.parse(output_content)
    return output


In [3]:
# Run
for claim in db.get_all_claims()[:-1]:
    
    #[0]: claims_id
    #[1]: code (from CARDS spreadsheet)
    #[2]: claim text
    #[3]: argument_validity {1:True, 0:False}
    argument = claim[2]
    claims_id = claim[0]
    print(claims_id, argument)


    output = first_chain(argument)
    print(output)
    output2 = second_chain(output, argument)
    print(output2)

    if isinstance(output2['valid'], str):
        output2['valid'] = PromptUtility.approximate_bool(output2['valid'])
        
    if output2['valid'] == False:
        output3 = third_chain(output, argument)
        output.update(output3)
        print(output3)

    output4s = dict()
    for k,premise in output.items():
        if k.lower().startswith('p') or k.lower().startswith('h'):
            output4 = fourth_chain(premise)
            output4s[k] = output4

    print(output4s)

    # insert validity:
    db.insert_validity(claims_id, output2['reason'], output2['valid'])

    # insert premises:
    for premise_identifier,premise in output.items():
        is_hidden = False
        if not premise_identifier.lower().startswith('a'):
            if premise_identifier.lower().startswith('h'):
                is_hidden = True
            premise_id = db.insert_premise(claims_id, premise, is_hidden, premise_identifier)
            # insert fallacy:
            for identifier,fallacies in output4s.items():
                if premise_identifier == identifier:
                    for fallacy in fallacies:
                        db.insert_fallacy(premise_id, fallacy)

1 Antarctic sea ice is increasing. Therefore global warming is not happening.
{'P1': 'Antarctic sea ice is increasing', 'C': 'Therefore global warming is not happening'}
{'valid': 'FALSE', 'reason': "This argument is logically invalid. The fact that Antarctic sea ice is increasing does not necessarily mean that global warming is not happening. Global warming refers to the overall increase in the Earth's average temperature, and the increase in Antarctic sea ice does not negate this phenomenon."}
{'HP': 'Antarctic sea ice is a reliable indicator of global warming'}
{'P1': ['Cherry Picking'], 'HP': ['No fallacy.']}
2 Parts of Greenland's ice sheet are not melting. Therefore global warming is not happening.
{'P1': "Parts of Greenland's ice sheet are not melting", 'C': 'Therefore global warming is not happening'}
{'valid': 'FALSE', 'reason': "This argument is logically invalid. The fact that parts of Greenland's ice sheet are not melting does not necessarily mean that global warming is not

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 29 Jun 2023 07:46:43 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7dec87082c36274a-ADL', 'alt-svc': 'h3=":443"; ma=86400'}.


{'P1': 'Current climate change is driven by natural factors', 'P2': 'Natural factors are currently having a cooling effect', 'C': 'If natural cooling continues, we will experience more cooling in the future'}
{'valid': 'TRUE', 'reason': 'This argument is logically valid. If it is true that current climate change is driven by natural factors and those natural factors are currently having a cooling effect, then it would follow that if natural cooling continues, we will experience more cooling in the future.'}
{'P1': ['False Analogy', 'Misrepresentation', 'Single Cause'], 'P2': ['No fallacy.']}
6 Cold weather events are occuring. Therefore global warming is not happening.
{'P1': 'Cold weather events are occurring', 'C': 'Therefore global warming is not happening'}
{'valid': 'FALSE', 'reason': 'This argument is logically invalid. The occurrence of cold weather events does not necessarily mean that global warming is not happening. Global warming refers to long-term trends in temperature, no

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 29 Jun 2023 07:54:55 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7dec931aad93274a-ADL', 'alt-svc': 'h3=":443"; ma=86400'}.


{'P1': ['No fallacy.'], 'P2': ['Misrepresentation'], 'HP': ['No fallacy.']}
19 Negative feedbacks in the climate system reduce warming. Therefore climate sensitivity to CO2 emissions is low.
{'P1': 'Negative feedbacks in the climate system reduce warming', 'C': 'Therefore climate sensitivity to CO2 emissions is low'}
{'valid': 'TRUE', 'reason': 'This argument is logically valid. If negative feedbacks in the climate system reduce warming, then it follows that the climate sensitivity to CO2 emissions is low.'}
{'P1': ['No fallacy.']}
20 Species have shown resilience to climate change either now or in the past. Therefore species can adapt to climate change.
{'P1': 'Species have shown resilience to climate change either now or in the past.', 'C': 'Therefore species can adapt to climate change.'}
{'valid': 'TRUE', 'reason': 'This argument is logically valid. If species have shown resilience to climate change either now or in the past, it follows that they can adapt to climate change.'}
{'P1

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'P1': ['No fallacy.'], 'P2': ['No fallacy.'], 'HP': ['No fallacy.']}
28 A single policy would have a negligible impact. Therefore we should not have the policy.
{'P1': 'A single policy would have a negligible impact', 'C': 'Therefore we should not have the policy'}
{'valid': 'TRUE', 'reason': 'This argument is logically valid. If it is true that a single policy would have a negligible impact, then it follows that we should not have the policy.'}
{'P1': ['No fallacy.']}
29 Climate action reduces wealth and wealth increases resilience to climate action. Therefore it is better to adapt to climate change than to mitigate it.
{'P1': 'Climate action reduces wealth', 'P2': 'Wealth increases resilience to climate action', 'C': 'It is better to adapt to climate change than to mitigate it'}
{'valid': 'FALSE', 'reason': 'The conclusion does not logically follow from the premises. While it is true that climate action reduces wealth and wealth increases resilience to climate action, it does not ne

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


{'P1': ['no fallacy'], 'P2': ['No fallacy.']}
32 Much of the public aren't convinced about the need for climate action. Therefore political action on climate change is too hard.
{'P1': "Much of the public aren't convinced about the need for climate action.", 'C': 'Political action on climate change is too hard.'}
{'valid': 'FALSE', 'reason': 'The conclusion does not logically follow from the premise. Just because much of the public is not convinced about the need for climate action does not mean that political action on climate change is too hard.'}
{'HP': 'If the public is not convinced about the need for climate action, it is difficult to implement political action on climate change.'}
{'P1': ['No fallacy.'], 'HP': ['no fallacy']}
33 Clean energy has costs on the economy and the environment. Therefore clean energy is harmful.
{'P1': 'Clean energy has costs on the economy and the environment.', 'C': 'Therefore clean energy is harmful.'}
{'valid': 'FALSE', 'reason': 'This argument is l

In [4]:
type(output)

dict

In [5]:
output

{'P1': 'Governing bodies and corporations act secretly on climate policy',
 'C': 'Climate policy is part of a conspiracy',
 'A': 'Governing bodies and corporations act secretly on climate policy. Therefore climate policy is part of a conspiracy.',
 'HP': 'If actions are done secretly, they must be part of a conspiracy'}

In [7]:
first_response_schemas = [
    ResponseSchema(name="P#", description="premise of the argument"),
    ResponseSchema(name="C", description="conclusion of the argument")]

first_output_parser = StructuredOutputParser.from_response_schemas(first_response_schemas)

In [8]:
first_output_parser.get_format_instructions()

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\\`\\`\\`json" and "\\`\\`\\`":\n\n```json\n{\n\t"P#": string  // premise of the argument\n\t"C": string  // conclusion of the argument\n}\n```'